In [1]:
import pandas as pd
import numpy as np
import warnings
import bisect
from matplotlib import pyplot as plt

In [253]:
# collect data
warnings.filterwarnings('ignore')
file_path = '/Users/William/Downloads/CRKLAGGED5.csv'
data_start = pd.read_csv(file_path)
#data_start = data_start.iloc[0:500]
years = list(range(2019,2020))

In [254]:
# transition matrix
def transition_matrix(df,year):
    
    ## only use the same two years for transitions and drop nans
    df = df[['CRK t-2','CRK t-1','year']]
    df = df.dropna()
    
    ## mandate transitions cannot go up
    df = df[(df['CRK t-2'] >= df['CRK t-1']) ]#& (df['CRK t-2'] - df['CRK t-1'] <= 1)]
    df[['CRK t-2','CRK t-1','year']] = df[['CRK t-2','CRK t-1','year']].applymap(lambda x: round(x*2)/2)
    
    ## initialize possible states and Markov Chain
    states = np.arange(0,10.5,.5).tolist()
    tmat = pd.DataFrame(index=states,columns=states)
    
    ## count number of transitions from one state to the next
    for s1 in states:
        
        for s2 in states:
            
            ## only use data from up to 15 years prior
            temp = df[(df['CRK t-2'] == s1) & (df['CRK t-1'] == s2) & (df['year'] < year) & (df['year'] > year - 15)]
            tmat[s2][s1] = temp.shape[0] 
    
    tmat.to_csv('/Users/William/Documents/A HS Spring 2020/CEE 8200/check.csv')
    
    ## convert to cumulative density
    tmat['sum'] = tmat.sum(axis=1)
    tmat['sum'] = tmat['sum'].replace(0,1)
    tmat = tmat[tmat.columns[0:-1]].div(tmat['sum'],axis=0)
    tmat = tmat.cumsum(axis=1)
    
    return tmat

In [266]:
def run_sims(data,cdfs):
    ## number of simulations and time horizon
    runs = 50
    win = 5
#     print(len(data))
#     print(len(data['PID'].unique()))
    
    for r,road in enumerate(data['PID']):
        
        if data['CYCLE'].iloc[r] <= 2:
            cdf = cdfs[0]
        elif data['CYCLE'].iloc[r] in [3,4]:
            cdf = cdfs[1]
        else:
            cdf = cdfs[2]
    
        ## calculate 5 year forecast for every single road
        ## run for the desired number of simulations
        for run in range(runs):

            forecasts = []
            s1 = data['t0'].iloc[r]
            
            for t in range(0,5):
                s2 = monte_carlo(s1,cdf)
                forecasts.append(s2)
                s1 = s2
            data['Forecasted Paths'].iloc[r].append(forecasts)
            
        ## calculate point estimate for each state
        cols = ['Et1','Et2','Et3','Et4','Et5']
        for t in range(0,5):
            
            col = cols[t]
            
            ## elements in forecast list are organized from
            ## closest to furthest forecast, so make a list
            ## that pulls elements based on their index
            sims = data['Forecasted Paths'].iloc[r]
            item = [sims[s][t] for s in range(len(sims))]
            value = np.mean(item)
            
            data[col].iloc[r] = value

    return data

In [267]:
def monte_carlo(state,cdf):
    
    U = np.random.uniform()
    row = np.array(cdf.loc[state])
    
    if sum(row) == 0:
        prediction = max(0,state-.5)
    else:
        idx = bisect.bisect(row,U)
        prediction = cdf.columns[idx]
    
    return prediction

In [268]:
def find_failures(data):
    
    ## initialize list that tracks evaluatbale roads
    ## at each time window, length of items in list
    ## should be monotonically decreasing
    t = []
    cols = ['t0','t1','t2','t3','t4','t5']
    est = ['Et0','Et1','Et2','Et3','Et4','Et5']
    mid = data
    for i in range(5):
        
        ## evaluatable roads should not be failing in the current year
        ## and not be resurfaced in the next year
        tn = mid[(mid[cols[i]] >= 6.5) & (mid[cols[i]] >= mid[cols[i+1]])] 
                 #& (mid[cols[i]] - mid[cols[i+1]] <= 1.5)]
        t.append(tn)
        mid = tn
    
    ## create column for failure state or not
    base_file = '/Users/William/Documents/A HS Spring 2020/CEE 8200/'
    tag = '.csv'
    casts = []
    keep = ['PID','Year','Years Ahead']
    for i,cast in enumerate(t):
        
        true = cols[i+1]
        estimated = est[i+1]
        
        mid = keep
        mid.append(true)
        mid.append(estimated)
        cast['Fail'] = cast[true].apply(lambda x: int((x<6.5)))
        cast['EFail'] = cast[estimated].apply(lambda x: int((x<6.5)))
        
        add = 'Year' + str(i+1) +'Forecast'
        filepath = base_file + add + tag
        cast.to_csv(filepath)
        casts.append(cast)

    return casts

In [269]:
def evaluate(data,n):
    col1 = 't' + str(n+1)
    col2 = 'Et' + str(n+1)
    
    data['State Error'] = data[col1] - data[col2]
    data['State Error'] = data['State Error'].apply(lambda x: abs(x))
    
    sample = len(data)
    state_error = data[data[col1] != data[col2]]
    pf_error = data[data['Fail'] != data['EFail']]
    
    fp = data[(data['Fail'] ==0) & (data['EFail'] ==1)]
    fn = data[(data['Fail'] ==1) & (data['EFail'] ==0)]
    
    pos = data[data['Fail'] == 1]
    neg = data[data['Fail'] == 0]
    
    bad = data[data['State Error'] > 3]
    
    mat = pd.DataFrame(columns=['Pos','Neg'],index=['Pred Pos','Pred Neg'])
    mat['Pos'].loc['Pred Pos'] = len(data[(data['Fail'] ==1) & (data['EFail'] ==1)])
    mat['Pos'].loc['Pred Neg'] = len(data[(data['Fail'] ==1) & (data['EFail'] ==0)])
    mat['Neg'].loc['Pred Pos'] = len(data[(data['Fail'] ==0) & (data['EFail'] ==1)])
    mat['Neg'].loc['Pred Neg'] = len(data[(data['Fail'] ==0) & (data['EFail'] ==0)])
    
    print('Year t+',n+1)
    print('Sample Size:',sample)
    print('Average State Error:',round(data['State Error'].mean(),2))
    print('Pass/Fail Error Rate:',round(len(pf_error)/sample,2))
    print('False Negative Rate:',round(len(fn)/len(pos),2))
    print('False Positive Rate:',round(len(fp)/len(neg),2))
    #print(bad[['PID','t0',col1,col2]])
    print(mat)
    print('')

In [276]:
def main(data,years):
    
    ## drop all nans and round states to the nearest .5
    print(len(data))
    sub = list(data.columns[24:])
    sub.append('RCILM')
    data = data.dropna(subset=sub)
    print(len(data))
    data[data.columns[24:]] = data[data.columns[24:]].applymap(lambda x: round(x*2)/2)
    trans_data = data#.drop_na()
    
    ## reduce data to necessary columns, rename, and reorder them
    ## from earliest year to latest
    data = data[['PID','year','CYCLE','RCILM','CRK', 'CRK t-1', 'CRK t-2','CRK t-3', 'CRK t-4', 'CRK t-5']]
    
    data = data.rename(columns={'PID':'PID','year':'year','CYCLE':'CYCLE','RCILM':'RCILM','CRK t-5':'t0',
                         'CRK t-4':'t1','CRK t-3':'t2',
                         'CRK t-2':'t3','CRK t-1':'t4',
                         'CRK':'t5'})
    cols = ['PID','year','CYCLE','RCILM','t0','t1','t2','t3','t4','t5']
    data = data[cols]
    

    ## develop forecasts for each target year
    for year in years:
        year_data = data[data['year'] == year]
        
        ## initialize columns that will hold year estimates
        ## and that will store list of each 5-year forecasts
        year_data['Et1'] = np.nan
        year_data['Et2'] = np.nan
        year_data['Et3'] = np.nan
        year_data['Et4'] = np.nan
        year_data['Et5'] = np.nan
        year_data['Forecasted Paths'] = [[] for i in range(len(year_data))]
        
        
        ## calculate transition matrices, 
        ## input year is 5 years ahead of target year
        trans_year = year-5
        t_you = trans_data[trans_data['CYCLE'] <= 2]
        t_mid = trans_data[(trans_data['CYCLE'] >2) & (trans_data['CYCLE'] <=4)]
        t_old = trans_data[trans_data['CYCLE'] > 4]
        
        cdf_you = transition_matrix(t_you,year)
        cdf_mid = transition_matrix(t_mid,year)
        cdf_old = transition_matrix(t_old,year)
        cdf = [cdf_you,cdf_mid,cdf_old]
        
        ## run simulation and calculate point estimates
        forecasts = run_sims(year_data,cdf)
        print(forecasts)
        
        ## calculate faiures
        evals = find_failures(forecasts)
        
        ## evaluate performance
        for n,item in enumerate(evals):
            evaluate(item,n)
        
    
main(data_start,years) 

91428
90734
        PID  year  CYCLE   RCILM    t0    t1    t2    t3    t4    t5   Et1  \
14        0  2019      3   0.390  10.0  10.0  10.0  10.0   9.0   9.0  9.88   
27        1  2019      4   1.861   8.5   7.5   7.5   7.5   6.0   6.0  8.17   
40        2  2019      3   0.204  10.0  10.0  10.0  10.0  10.0  10.0  9.88   
57        6  2019      2   4.790  10.0  10.0  10.0  10.0   7.5   7.0  9.68   
69        7  2019      2   8.160   7.0   7.0   6.5   6.5   4.5   4.5  6.84   
...     ...   ...    ...     ...   ...   ...   ...   ...   ...   ...   ...   
91372  7744  2019      3  32.200   9.0   8.0   8.0   6.5   6.5   5.5  8.75   
91383  7745  2019      4   7.518  10.0  10.0  10.0   8.5   8.5   8.0  9.88   
91398  7746  2019      3  30.200   9.5   9.5   9.5   9.0   8.0   8.0  9.09   
91415  7747  2019      3  32.200   9.5   9.5   9.0   7.5   6.5   5.5  9.06   
91427  7748  2019      4   7.518  10.0  10.0  10.0   9.5   9.5   9.5  9.81   

        Et2   Et3   Et4   Et5  \
14     9.71  9.51 

In [262]:
print(len(data_start))
data2 = data_start.dropna()
print(len(data2))
data3 = data_start[data_start['is_decreasing'] == True]
print(len(data3))

91428
28393
86541
